# Part 1: MLE + Bayes

In [1]:
import os
from collections import defaultdict

import numpy as np
from PIL import Image

## Working with styles

### Assigning different styles and their probs

In [2]:
styles = {
    "прическа": [
        "нет волос",
        "длинные в пучок",
        "длинные волнистые",
        "длинные прямые",
        "короткая волнистые",
        "короткая прямые",
        "короткая курчавые",
    ],
    "цвет волос": [
        "черный",
        "блонд",
        "каштановый",
        "пастельный розовый",
        "рыжий",
        "серебристо серый",
    ],
    "аксесуар": [
        "нет очков",
        "круглые очки",
        "солнцезащитные очки",
    ],
    "одежда": [
        "худи",
        "комбинезон",
        "футболка с круглым вырезом",
        "футболка с V-вырезом",
    ],
    "цвет одежды": [
        "черный",
        "синий",
        "серый",
        "зеленый",
        "оранжевый",
        "розовый",
        "красный",
        "белый",
    ],
}

styles_count = {
    "прическа": [7, 0, 1, 23, 1, 11, 7],
    "цвет волос": [
        7,
        6,
        2,
        3,
        8,
        24,
    ],
    "аксесуар": [
        11,
        22,
        17,
    ],
    "одежда": [
        7,
        18,
        19,
        6,
    ],
    "цвет одежды": [4, 5, 6, 8, 6, 8, 7, 6],
}

### MLE probs function

In [3]:
def get_mle_probs(arr):
    d = sum(arr) + len(arr)
    return [(el + 1) / d for el in arr]

### Recursive function to get every combination possible

In [4]:
def generate_bayes_dist(
    _dict: dict, keys: list, ck: int, cumstr: str, p: float, res: dict
):
    k = keys[ck]
    for v in _dict[k]:
        if k != keys[-1]:
            generate_bayes_dist(
                _dict,
                keys,
                ck + 1,
                v[0] if not ck else f"{cumstr}, {v[0]}",
                v[1] if not ck else p * v[1],
                res,
            )
        else:
            res[f"{cumstr}, {v[0]}"] = p * v[1]

### Using `np.random.choice` to get random example from result with respective probs

In [5]:
def get_random_style(_dict: dict):
    ch = np.random.choice(list(_dict.keys()), p=list(_dict.values()))
    return ch, _dict[ch]

In [6]:
styles_count = {k: get_mle_probs(v) for k, v in styles_count.items()}

styles_merged = {k: list(zip(styles[k], styles_count[k])) for k in styles_count}

In [7]:
k = list(styles_count.keys())

pix = {}
generate_bayes_dist(styles_merged, k, 0, "", 0, pix)

### Click to get random style!

In [8]:
get_random_style(pix)

('короткая прямые, серебристо серый, нет очков, комбинезон, синий',
 0.0007745453418843139)

## Working with images

In [48]:
SOURCE_IMGS_DIR = "avatars"
GEN_IMGS_DIR = "avatars_gen"
IMG_H = 560
IMG_W = 528
EXAMPLES = 5

In [ ]:
imgs = [os.path.join(SOURCE_IMGS_DIR, el) for el in os.listdir(SOURCE_IMGS_DIR)]

### Loading images and getting pixels info

In [44]:
imgs = [os.path.join(SOURCE_IMGS_DIR, el) for el in os.listdir(SOURCE_IMGS_DIR)]

pix = defaultdict(dict)
for img in imgs:
    pimg = Image.open(img)
    for x in range(pimg.width):
        for y in range(pimg.height):
            r, g, b, _ = pimg.getpixel((x, y))

            if not pix[f"{x}_{y}"].get("r"):
                pix[f"{x}_{y}"]["r"] = []
            if not pix[f"{x}_{y}"].get("g"):
                pix[f"{x}_{y}"]["g"] = []
            if not pix[f"{x}_{y}"].get("b"):
                pix[f"{x}_{y}"]["b"] = []

            pix[f"{x}_{y}"]["r"].append(r)
            pix[f"{x}_{y}"]["g"].append(g)
            pix[f"{x}_{y}"]["b"].append(b)

for p in pix:
    pix[p]["r"] = (pix[p]["r"], get_mle_probs(pix[p]["r"]))
    pix[p]["g"] = (pix[p]["g"], get_mle_probs(pix[p]["g"]))
    pix[p]["b"] = (pix[p]["b"], get_mle_probs(pix[p]["b"]))

In [45]:
def get_random_pix(val_probs: list):
    return np.random.choice(val_probs[0], p=val_probs[1])

### Generating new avatars!

In [49]:
os.makedirs(GEN_IMGS_DIR, exist_ok=True)

for i in range(EXAMPLES):
    res = Image.new("RGB", (IMG_W, IMG_H))
    for x in range(pimg.width):
        for y in range(pimg.height):
            res.putpixel(
                (x, y),
                (
                    get_random_pix(pix[f"{x}_{y}"]["r"]),
                    get_random_pix(pix[f"{x}_{y}"]["g"]),
                    get_random_pix(pix[f"{x}_{y}"]["b"]),
                ),
            )

    res.save(os.path.join(GEN_IMGS_DIR, f"img_{i}.png"), format="png")